# Goals
- train predictor on training trusted and untrusted sets 
- use confidence as tampering score

In [ ]:
from cupbearer import data, detectors, models, scripts, tasks, utils
from torch.utils.data import DataLoader

# Train Measurement Predictor

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["CUDA_VISIBLE_DEVICES"]

In [ ]:
train_data = data.TamperingDataset("diamonds", train=True)
val_data = data.TamperingDataset("diamonds", train=False)

In [ ]:
transformer, tokenizer, emb_dim, max_len = models.transformers_hf.load_transformer(
    "pythia-14m"
)
model = models.TamperingPredictionTransformer(
        model=transformer,
        tokenizer=tokenizer,
        embed_dim=emb_dim,
        max_length=max_len,
        n_sensors=3,
    )

In [ ]:
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
num_epochs = 1
total_steps = num_epochs * len(train_loader)
lr = 2e-5
scripts.train_classifier( # NOTE: - paper uses 64 warmup steps, but seems hard
    path=(classifier_path := utils.log_path("logs/tampering/predictor")),
    model=model,
    train_loader=train_loader,
    task="multilabel",
    num_labels=4,
    val_loaders=DataLoader(val_data, batch_size=1, shuffle=False),
    lr=lr,
    lr_scheduler_conf={
        "lr_warmup_steps": 64,
        "total_steps": total_steps,
        "lr": lr
    },
    lr_scheduler_builder=scripts.lr_scheduler.CosineWarmUpBuilder,
    max_epochs=num_epochs,
)